# Modelling

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.integrate import solve_ivp


def integrate(model, y0, parameters, t_end, n_points=100):
    sol = solve_ivp(
        lambda _, y, *args: model(*y, *args),
        args=parameters,
        y0=list(y0.values()),
        t_span=(0, t_end),
        t_eval=np.linspace(0, t_end, n_points),
        method="LSODA",
    )
    return pd.DataFrame(sol.y.T, index=sol.t, columns=y0.keys())

In [ ]:
def exponential_growth(N, k):
    return k * N

df = integrate(
    exponential_growth,
    y0={"N": 1.0},
    parameters=(1,),
    t_end=5,
)

In [ ]:
df

In [ ]:
df.plot()